In [1]:
%%writefile 2.txt
first
second
third
fourth

Writing 2.txt


In [3]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 199 kB 53.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=b1d01332ca694a32cf2436fae9ba11a223243b69856c5b854321e3f452cd1f8f
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [4]:
from pyspark import SparkContext

In [5]:
sc=SparkContext()

In [9]:
text=sc.textFile('2.txt')

In [10]:
text

2.txt MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0

In [11]:
words=text.map(lambda line: line.split())

In [13]:
words.collect()

[['first'], ['second'], ['third'], ['fourth']]

In [15]:
text.flatMap(lambda line: line.split()).collect()

['first', 'second', 'third', 'fourth']

In [16]:
%%writefile s.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
206       10/19/2017      202       CA       131          500.00
203       10/17/2017      101       NY       173          750.00
205       10/19/2017      202       TX       121          200.00

Writing s.txt


In [25]:
sec=sc.textFile('s.txt')

In [26]:
sec

s.txt MapPartitionsRDD[11] at textFile at NativeMethodAccessorImpl.java:0

In [28]:
sec.take(2) #head of data

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00']

In [30]:
sec.map(lambda line:line.split()).take(3)

[['#EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00']]

In [33]:
clean=sec.map(lambda line:line[1:] if line[0]=='#' else line)

In [34]:
clean=clean.map(lambda line:line.split())

In [35]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [39]:
pairs=clean.map(lambda lst:(lst[3],lst[-1]))

In [42]:
rekey=pairs.reduceByKey(lambda amt1,amt2:float(amt1)+float(amt2))

In [43]:
rekey.collect()

[('State', 'Amount'), ('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [44]:
# Grab state and amounts
# Add them
# Get rid of ('State','Amount')
# Sort them by the amount value
clean.map(lambda lst: (lst[3],lst[-1])).reduceByKey(lambda amt1,amt2 : float(amt1)+float(amt2)).filter(lambda x: not x[0]=='State').sortBy(lambda stateAmount: stateAmount[1], ascending=False).collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]

In [45]:
x = ['ID','State','Amount']

In [46]:
def func1(lst):
  return lst[-1]

In [47]:
def func2(id_st_amt):
  (id,st,amt)=id_st_amt
  return amt

In [49]:
func1(x)

'Amount'

In [50]:
func2(x)

'Amount'